# ElasticSearch

In this notebook we'll implement a simple search using ElasticSearch.
Because ES does not support polish language natively, we'll use polish analyzer plugin to add polish language support.

There are many ways we can add this support. The simplest is to set a polish analyzer to selected properties. 

We can also set a custom tokenizers. That would let us have more control over the way the fields are tokenized. Some words like "e-mail" might be split into "e" and "mail" which is not something we want. 

There is also a plugin created by Allegro, which might work better for some parts. It's also possible to add a dictionary of synonyms.

In later notebooks we'll look at how each of this methods works and what result can we achieve.

In [81]:
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')

mapping = {
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "analyzer": "polish"
            }
        }
    }
}

es.options(ignore_status=404).indices.delete(index='test_index')
es.options(ignore_status=400).indices.create(index='test_index', body=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

## Analyze the query

In [76]:
text = "Matematyka jest super! Bardzo mi się podobają trójkąty i równania."

analysis = es.indices.analyze(index='test_index', body={
    'analyzer': 'polish',
    'text': text
})

for token in analysis['tokens']:
    print(token['token'])

matematyka
super
podobać
trójkąt
równania


## Add data to elastic search

In [78]:
sentences = [
    "Historia Polski jest bardzo ciekawa. Mieszko I był pierwszym władcą Polski. W roku 966 przyjął chrzest. W ten sposób Polska stała się krajem chrześcijańskim.",
    "Maria Curie-Skłodowska była wybitną polską naukowczynią. Otrzymała dwie Nagrody Nobla. Była pierwszą kobietą, która otrzymała tę nagrodę. Chemia była jej pasją. Odkryła pierwiastki promieniotwórcze.",
    "Trygonometria jest działem matematyki. Zajmuje się ona badaniem zależności między bokami i kątami trójkątów. Trygonometria jest bardzo ważna w nawigacji. Dzięki niej możemy określić położenie statku na morzu.",
]

for i, sentence in enumerate(sentences):
    es.index(index='test_index', id=i, body={'content': sentence})

response = es.search(index='test_index', body={"query": {"match_all": {}}})

# For each document in the response, analyze its 'content' field
for hit in response['hits']['hits']:
    document = hit['_source']
    content = document['content']
    
    # Analyze the content
    analysis = es.indices.analyze(index='test_index', body={
        'analyzer': 'polish',
        'text': content
    })
    
    # Print the tokens
    print("Tokens for document", hit['_id'], ":")
    for token in analysis['tokens']:
        print(token['token'])
    print()


Tokens for document 0 :
historia
polski
ciekawy
mieszko
pierwszey
władca
polski
966
przyjąć
chrzesć
polski
stała
kraj
chrześcijański

Tokens for document 1 :
maria
curia
skłodowski
wybitny
polski
naukowczynić
otrzymać
dwie
nagroda
nobl
pierwy
kobieta
otrzymać
nagroda
chemia
pasja
odkryć
pierwiastek
promieniotwórczy

Tokens for document 2 :
trygonometria
działem
matematyka
zajmować
on
badać
zależność
międyć
bok
kąt
trójkąt
trygonometria
ważny
nawigacja
dzięk
móc
określić
położen
statek
morze



## Query the data

In [79]:
query = {
    "query": {
        "match": {
            "content": {
                "query": "matematyka",
                "analyzer": "polish"
            }
        }
    }
}

results = es.search(index='test_index', body=query)

print('Liczba wyników:', results['hits']['total']['value'])
for hit in results['hits']['hits']:
    print('Dokument:', hit['_source']['content'])
    print('Wynik:', hit['_score'])
    print()

Liczba wyników: 1
Dokument: Trygonometria jest działem matematyki. Zajmuje się ona badaniem zależności między bokami i kątami trójkątów. Trygonometria jest bardzo ważna w nawigacji. Dzięki niej możemy określić położenie statku na morzu.
Wynik: 0.9768399

